In [1]:
import os
cwd = os.getcwd().replace('/vectorbt', '')
os.chdir(cwd)

In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt
vbt.settings.set_theme('dark')


### Upload dataset

In [3]:
dataset = pd.read_csv('data/dataset_5m_180day_tickers.csv' , index_col=0)
dataset.index = pd.to_datetime(dataset.index)

### Resample dataset

In [4]:
close = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('5Min')\
    .last()\
    .unstack()\
    .T

close_15m = dataset.copy().\
    groupby('symbol')['close']\
    .resample('15Min')\
    .last()\
    .unstack()\
    .T
close_1h = dataset.copy().\
    groupby('symbol')['close']\
    .resample('60Min')\
    .last()\
    .unstack()\
    .T

close_1d = dataset.copy().\
    groupby('symbol')['close']\
    .resample('1d')\
    .last()\
    .unstack()\
    .T

#### Working with indicators
* Create indicators for each timeframe
* Join tables using *align* method from pandas

In [5]:
rsi_15m = vbt.RSI.run(close_15m, window=14, short_name='RSI').rsi
rsi_15m, _ = rsi_15m.align(close, broadcast_axis=0,
                           method='ffill', join='right')

rsi_1h = vbt.RSI.run(close_1h, window=14, short_name='RSI').rsi
rsi_1h, _ = rsi_1h.align(close, broadcast_axis=0, method='ffill', join='right')

rsi_1d = vbt.RSI.run(close_1d, window=14, short_name='RSI').rsi
rsi_1d, _ = rsi_1d.align(close, broadcast_axis=0, method='ffill', join='right')

#### Define trading logic
* Create datafrmame shape with signal 0
* Define signal of trading logic

In [6]:
signal = rsi_15m.copy().applymap(lambda x: 0)
signal[
    (rsi_15m < 20) &
    (rsi_1h < 20) &
    (rsi_1d < 30)
] = 1
signal[
    (rsi_15m > 80) &
    (rsi_1h > 80) &
    (rsi_1d > 70)
] = -1


#### Backtesting strategy

In [7]:
entries = np.where(signal == 1, True, False)
exits = np.where(signal == -1, True, False)

entries_short = np.where(signal == -1, True, False)
exits_short = np.where(signal == 1, True, False)
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    short_entries=entries_short,
    short_exits=exits_short,
    fees=0.001  
)
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol',
                       yaxis_title='Max total return', height=400, width=1700)


FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

In [8]:
pf.total_return()

symbol
ADAUSDT     -0.265477
ALGOUSDT     2.073633
ATOMUSDT    -0.370375
BCHUSDT     -0.290060
BNBUSDT      0.038982
BTCUSDT     -0.313265
DOGEUSDT    -0.052635
DOTUSDT     -0.445354
ETCUSDT     -0.323576
ETHUSDT      0.046184
FILUSDT     -0.369928
LINKUSDT    -0.274348
LTCUSDT     -0.114902
LUNAUSDT    -0.394965
MATICUSDT    0.151215
SOLUSDT     -0.292027
XLMUSDT     -0.361588
XRPUSDT     -0.389702
Name: total_return, dtype: float64

In [199]:
data = vbt.BinanceData.download(['ALGOUSDT'], start='180 days ago', end='now UTC', interval='5m')


0it [00:00, ?it/s]

In [200]:
close = data.get()['Close']


close_15m = close.resample('15Min').last()
close_1h = close.resample('60Min').last()
close_1d = close.resample('1d').last()

rsi_15m = vbt.RSI.run(close_15m, window=14, short_name='RSI').rsi
rsi_15m, _ = rsi_15m.align(close, broadcast_axis=0,
                            method='ffill', join='right')   
rsi_1h = vbt.RSI.run(close_1h, window=14, short_name='RSI').rsi
rsi_1h, _ = rsi_1h.align(close, broadcast_axis=0, method='ffill', join='right')

rsi_1d = vbt.RSI.run(close_1d, window=14, short_name='RSI').rsi
rsi_1d, _ = rsi_1d.align(close, broadcast_axis=0, method='ffill', join='right')

signal = pd.DataFrame(close.copy()).applymap(lambda x: 0)
signal[
    (rsi_15m < 20) &
    (rsi_1h < 20) &
    (rsi_1d < 30)
] = 1
signal[
    (rsi_15m > 80) &
    (rsi_1h > 80) &
    (rsi_1d > 70)
] = -1

entries = np.where(signal == 1, True, False)
exits = np.where(signal == -1, True, False)

entries_short = np.where(signal == -1, True, False)
exits_short = np.where(signal == 1, True, False)
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    short_entries=entries_short,
    short_exits=exits_short,
    fees=0.001  
)
pf.stats()

/tmp/ipykernel_16035/607383427.py:42: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7f57dc694b80>. Pass column to select a single column/group.



Start                         2021-11-08 16:00:00+00:00
End                           2022-05-07 15:55:00+00:00
Period                                180 days 00:00:00
Start Value                                       100.0
End Value                                    224.461222
Total Return [%]                             124.461222
Benchmark Return [%]                         -62.505053
Max Gross Exposure [%]                            100.0
Total Fees Paid                                1.198113
Max Drawdown [%]                              44.937671
Max Drawdown Duration                  37 days 02:55:00
Total Trades                                        5.0
Total Closed Trades                                 4.0
Total Open Trades                                   1.0
Open Trade PnL                                  2.50047
Win Rate [%]                                      100.0
Best Trade [%]                                45.545987
Worst Trade [%]                                6